In [1]:
#get library
import numpy as np
import pandas as pd

In [2]:
test_set = pd.read_csv("testing_set.csv")

### Get rating of each user

In [7]:
def getRatingVectorOfNthUser(n:int,user_need_to_hide):
    #get the rating vector of n-th user and the min and max anime_id
    column_n = str(n) 
    rating = test_set[[column_n]].values
    not_minus1_sum,count = 0,0
    for i in rating:
        if i[0] != -1 and i[0] != 0:
            not_minus1_sum += i[0]
            count += 1
    if count != 0:
        mean = round(not_minus1_sum/count,2)
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = mean
    else:
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = 5
        
    #normalize the rating value
    del not_minus1_sum,count
    #return 70% of rating vector if source is test else 100%
    sum,count = 0,0
    for r in rating:
        if r[0] != 0:
            sum += r[0]
            count += 1
    mean = round(sum/count,2)
    for i in range(len(rating)):
        if rating[i][0] != 0:
            rating[i][0] = rating[i][0] - mean
    if user_need_to_hide == n:
        select = (count*7)//10 #get 70% of the rating value when we need to hide the choosen user
        temp,index,true_rating = 0,[],[]
        for i in range (len(rating)):
            if rating[i][0] != 0:
                if temp >= select:
                    true_rating.append(rating[i][0])
                    index.append(i)
                    rating[i][0] = 0
                temp += 1
        del temp,select
        return rating,index,true_rating

    return rating

print(getRatingVectorOfNthUser(37020,37020))

(array([[ 1.57],
       [-0.43],
       [ 1.57],
       ...,
       [ 0.  ],
       [ 0.  ],
       [ 0.  ]]), [7325, 7342, 7503, 7537, 7693, 7712, 7747, 7808, 7889, 7959, 7960, 7964, 7975, 8053, 8220, 8323, 8329, 8380, 8407, 8419, 8440, 8450, 8504, 8526, 8560, 8595, 8683, 8781, 8856, 8865, 9014, 9152, 9222, 9245, 9271, 9418], [-1.4299999999999997, -1.4299999999999997, 0.5700000000000003, -0.4299999999999997, 1.5700000000000003, 0.5700000000000003, 1.5700000000000003, 0.5700000000000003, 1.5700000000000003, 0.5700000000000003, 0.5700000000000003, -1.4299999999999997, 0.5700000000000003, -0.4299999999999997, -1.4299999999999997, -3.4299999999999997, -3.4299999999999997, -0.4299999999999997, -0.4299999999999997, -1.4299999999999997, 1.5700000000000003, -1.4299999999999997, 1.5700000000000003, -0.4299999999999997, -0.4299999999999997, 0.5700000000000003, 1.5700000000000003, -0.4299999999999997, -1.4299999999999997, 0.5700000000000003, -1.4299999999999997, -5.43, -1.4299999999999997, -1.42

### DECOMPOSE THE MATRIX INTO 3 SUBMATRIX $U$,$Sigma$,$V^T$
- The current size of the testing matrix is 9724x1001

In [19]:
A = test_set.values
U,Sigma,VT = np.linalg.svd(A)
print(np.round(U,decimals=2))
print(np.round(Sigma,decimals=1))
print(np.round(VT,decimals=2))

[[-0.   -0.05  0.1  ... -0.   -0.   -0.  ]
 [-0.   -0.02  0.05 ... -0.   -0.   -0.  ]
 [-0.   -0.04  0.07 ...  0.    0.    0.  ]
 ...
 [-0.02  0.01  0.   ...  1.   -0.   -0.  ]
 [-0.02  0.01  0.   ... -0.    1.   -0.  ]
 [-0.02  0.01  0.   ... -0.   -0.    1.  ]]
[1.5105602e+06 9.8240000e+02 4.3350000e+02 ... 6.0000000e-01 6.0000000e-01
 4.0000000e-01]
[[-1.    0.   -0.   ... -0.   -0.   -0.  ]
 [ 0.    0.   -0.04 ... -0.01 -0.   -0.  ]
 [ 0.   -0.    0.03 ...  0.01 -0.    0.01]
 ...
 [ 0.   -0.7   0.   ...  0.   -0.    0.  ]
 [-0.    0.29 -0.   ... -0.   -0.   -0.  ]
 [-0.   -0.06  0.   ... -0.    0.    0.  ]]


### Hide 70% of a rating column in A matrix

Singular Value Decomposition (SVD) has an important property that makes it particularly interesting for our application. SVD provides the best low-rank linear approximation of the original matrix $A$. It is possible to retain only $k$ singular values by discarding others, resulting in a reduced matrix $S_k$. Since the singular values in $S$ are sorted such that $s_1 \geq s_2 \geq \ldots \geq s_r$, the reduction process involves retaining the first $k$ singular values. Matrices $U$ and $V$ are also reduced to produce matrices $U_k$ and $V_k$, respectively. $U_k$ is obtained by removing $(r - k)$ columns from matrix $U$, while $V_k$ is obtained by removing $(r - k)$ rows from matrix $V$. When these three reduced matrices are multiplied, we obtain a matrix $A_k$ given by $A_k = U_k S_k V_k^T$. This $A_k$ is a rank-$k$ matrix that provides the closest approximation to the original matrix $A$.



In [ ]:
print(U.shape)
print(VT.shape)
print(Sigma.shape)
print(A.shape)

## Prediction generation using SVD. 
Once the $m \times n$ ratings matrix $R$ is decomposed and reduced into three SVD component matrices with $k$ features $U_k$, $S_k$, and $V_k$, prediction can be generated from it by computing the cosine similarities (dot products) between m pseudo-customers $U_k \cdot \sqrt{S_k}^T$ and $n$ pseudo-products $\sqrt{S_k} \cdot V_k^T$ . In particular, the prediction score $P_{i,j}$ for the $i$-th customer on the $j$-th product is obtained by adding the row average $r_i$ to the similarity. Formally, $P_{i,j} = r_i + U_k \cdot \sqrt{S_k}^T(i) \cdot \sqrt{S_k} \cdot V_k^T(j)$. Once the SVD decomposition is done, the prediction generation process involves only a dot product computation, which takes $O(1)$ time, since $k$ is a constant.


In [24]:
print(*Sigma)

1510560.234569261 982.43265773739 433.5485183409572 354.4410842517081 324.84713336980735 303.6486368265189 285.60052879827117 258.6387146912024 257.12158229228 228.58471430398154 225.09629467709453 214.0966901781333 210.55722717497324 205.68107564774894 198.72514690904956 192.4376996763176 190.5062332360081 184.16328229580506 183.69434034007952 179.98303035007686 177.71275324972657 175.34703911133897 173.85619480519102 169.95847153575855 166.79292443173293 166.40837529618332 164.58580668552742 162.55325476867444 162.4166538974714 159.57909889018302 158.13925735691288 157.3889625819556 154.05149670560232 153.62364267891581 151.18109398417295 150.27925615096365 149.4732470008466 148.32045100549473 147.19463722113198 146.58231713728784 145.18182884871246 144.98041303793278 143.39981231229245 142.277441125817 142.00595856497964 140.80302777359063 140.61327162760384 138.95310890159652 138.58364381425483 137.3773491493186 136.61370115356374 135.6661955337419 135.64023125064094 134.2526392584

## Prediction formula of normalized utility matrix $A$

$$ \hat{r}_{ij} = \sum_{k=1}^{k} u_{ik} \cdot v_{jk} \cdot \sigma_k $$

Where:
- $\hat{r}_{ij}$  is the predicted rating of user \( i \) to item \( j \).
-  $u_{ik}$  is the  $(i,k)$ - th element of matrix $U$.
- $v_{jk}$  is the $(j,k)$ - th element of matrix $V^T$.
- $\sigma_k$ is the $k$-th singular value.




